# 优化器类

In [66]:
from abc import abstractmethod, ABC
import numpy as np

## Foundation

### Tensor

In [67]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = 0
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def size(self):
        return len(self.data)

    def __str__(self):
        return str(self.data)

### Base Layer

In [68]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### Base Loss Function

In [69]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

### Base Optimizer

In [70]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    def clear(self):
        for p in self.parameters:
            p.grad = 0

    @abstractmethod
    def step(self):
        pass

## Data

### Features and Label

In [71]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## Model

### Linear Layer

In [72]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad += p.grad * x.data
            self.bias.grad += np.sum(p.grad, axis=0)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### MSE Loss Function

In [73]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        mse = Tensor(np.mean(np.square(y.data - p.data)))

        def gradient_fn():
            p.grad += -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### SGD Optimizer

In [74]:
class SGDOptimizer(Optimizer):

    def step(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

## Configuration

### Learning Rate

In [75]:
LEARNING_RATE = 0.00001

## Testing

### Predicting

In [76]:
layer = Linear(feature.size(), 1)

prediction = layer(feature)

print(f'prediction: {prediction}')

prediction: [43.05]


### Calculating Loss

In [77]:
loss = MSELoss()

error = loss(prediction, label)

print(f'error: {error}')

error: 14871.802500000002


### Calculating Gradient and Updating Layer Parameters

In [78]:
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

optimizer.clear()
error.backward()
optimizer.step()

print(layer)

weight: [[0.5685359 0.641462 ]]
bias: [0.002439]
